In [2]:
import wikisearch
import wikipedia

In [ ]:
def search_subject(start,subs):
    

In [15]:
ww = wikipedia.page('Wonder Woman')
model = wikisearch.make_model_new([ww])

In [19]:
for link in ww.links

KeyError: "word 'Cheetah' not in vocabulary"

In [18]:
ww.links

['52 (comics)',
 '711 (Quality Comics)',
 'A.R.G.U.S.',
 'Abortion clinic',
 'Ace Magazines (comics)',
 'Ace the Bat-Hound',
 'Adam Hughes',
 'Aeaea',
 'Aegeus (comics)',
 'Aegis',
 'Air Wave',
 'Airboy',
 'Airman (character)',
 'Alan Scott',
 'Alberto Falcone',
 'Alex Ross',
 'Alfred Pennyworth',
 'Alien (creature in Alien franchise)',
 'All-American Publications',
 'All-Winners Squad',
 'All American Comics',
 'All Star Comics',
 'All Star Comics 8',
 'Alternative versions of Barbara Gordon',
 'Alternative versions of Batman',
 'Alternative versions of Joker',
 'Alternative versions of Lex Luthor',
 'Alternative versions of Robin',
 'Alternative versions of Supergirl',
 'Alternative versions of Superman',
 'Alternative versions of Wonder Woman',
 'Alura',
 'Amalthea (mythology)',
 'Amanda Waller',
 'Amazing-Man (Centaur Publications)',
 'Amazo',
 'Amazons (DC Comics)',
 'Amazons Attack!',
 'Ambush Bug: Year None',
 'American Ace',
 'American Comics Group',
 'American Crusader',
 'Ame